In [1]:
#Params
param_stationID = '260'
param_from_date = '2006'
param_to_date = '2024' 
param_vars = "TN, TX, RH"

param_stationID_spinup = '260'
param_from_date_spinup = '1995'
param_to_date_spinup = '2005' 
param_vars_spinup = "TN, TX, RH"

In [2]:
install.packages("readr")
install.packages("dplyr")
install.packages("tidyr")
install.packages("stringr")
install.packages("lubridate")
library(readr)
library(dplyr)
library(tidyr)
library(stringr)
library(lubridate)

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




In [9]:
#KNMI data retriever

#Parameter validation - check whether all parameters were filled in by user
if (param_stationID == "" ||
    param_from_date == "" ||
    param_to_date == "" ||
    length(param_vars) == 0) {
  stop("All parameters must be provided by the user.")
}

stationID  <- param_stationID
from       <- param_from_date
to         <- param_to_date

#Function to get the daily KNMI data, includes checks whether the dates are filled in correctly and logically 
get_daily_data <- function(stationID, from, to, vars) {
      vars <- unlist(strsplit(vars, ",\\s*"))
      
    #Date parsing - checking whether logical 
    if (!is.character(from) ||
      !is.character(to) ||
      stringr::str_length(from) %% 2 == 1 ||
      stringr::str_length(to) %% 2 == 1) {
    stop(
      "The values for 'from' and 'to' must be strings in the format 'YYYY', 'YYYYMM' or 'YYYYMMDD'."
    )
  }

  if (stringr::str_length(from) == 6) {
    from_date <- paste0(from, "01")
  } else if (stringr::str_length(from) == 4) {
    from_date <- paste0(from, "0101")
  } else {
    from_date <- from
  }

  if (stringr::str_length(to) == 8) {
    to_date <- to
  } else {
    if (stringr::str_length(to) == 6) {
      to <- paste0(to, "01")
    } else if (stringr::str_length(to) == 4) {
      to <- paste0(to, "1231")
    }

    to_date <- lubridate::ymd(to) %>%
      lubridate::ceiling_date(unit = "month") - 1

    to_date <- gsub("-", "", as.character(to_date))
  }

  if (as.numeric(to_date) < as.numeric(from_date)) {
    stop("'from' must be earlier than or equal to 'to'")
  }

  # Station validation - check whether station can be selected. The list contains all the KNMI weather stations that collect daily data 
  selectable_stations <- c("260", "275", "209", "210", "215", "225", "229", "235", "240", "242", "248", "249", "251", "257", "258", "260", "265", 
                           "267", "269", "270", "273", "275", "277", "278", "279", "280", "283", "285", "286", "290", "308", "310", "311", "312", 
                           "313", "315", "316", "319", "323", "324", "330", "331", "340", "343", "344", "348", "350", "356", "370", "375", "377",
                           "380", "391", "392")  
  stationID <- as.character(stationID)

  if (!stationID %in% selectable_stations) {
    stop(
      paste0(
        "Invalid stationID. Choose one of: ",
        paste(selectable_stations, collapse = ", ")
      )
    )
  }

#Build URL to retrieve the KNMI data from the website 
baseURL <- "https://www.daggegevens.knmi.nl/klimatologie/daggegevens"
  URL <- paste0(
    baseURL,
    "?start=", from_date,
    "&end=", to_date,
    "&stns=", stationID,
    "&ALL"
  )

  #Download data from KNMI 
  data_daily <- readr::read_csv(
    URL,
    col_names = FALSE,
    comment = "#",
    show_col_types = FALSE
  ) %>%
    dplyr::as_tibble()

  colnames(data_daily) <- c(
    "STN", "YYYYMMDD", "FG", "TG", "TN",
    "TX", "RH", "DDVEC", "UG", "UX", "UN"
  )

  # Select only requested variables as indicated when parameters were set 
  data_daily <- data_daily %>%
    tidyr::drop_na(dplyr::all_of(vars)) %>%
    select(dplyr::all_of(c("YYYYMMDD", vars)))

  # Scaling because KNMI works in 0.1
  if ("FG" %in% vars) data_daily$FG <- data_daily$FG / 10
  if ("TG" %in% vars) data_daily$TG <- data_daily$TG / 10
  if ("TN" %in% vars) data_daily$TN <- data_daily$TN / 10
  if ("TX" %in% vars) data_daily$TX <- data_daily$TX / 10
  if ("RH" %in% vars) data_daily$RH <- data_daily$RH / 10


  return(data_daily)
}

#Run the function for KNMI data and save it in an object
KNMI_daily_datafile <- ''
KNMI_daily_datafile <- get_daily_data(stationID = param_stationID,
  from      = param_from_date,
  to        = param_to_date,
  vars      = param_vars)


## For KNMI spinup data ##

#Parameter validation - check whether all parameters were filled in by user
if (param_stationID_spinup == "" ||
    param_from_date_spinup == "" ||
    param_to_date_spinup == "" ||
    length(param_vars_spinup) == 0) {
  stop("All parameters must be provided by the user.")
}

#Make sure function reads the spinup parameters 
stationID  <- param_stationID_spinup
from       <- param_from_date_spinup
to         <- param_to_date_spinup

#Function to get spinup data, includes checking whether the date is filled in correctly and logically 
get_spinup_data <- function(stationID, from, to, vars) {
      vars <- unlist(strsplit(vars, ",\\s*"))


#Date parsing - check whether date is logical
    if (!is.character(from) ||
      !is.character(to) ||
      stringr::str_length(from) %% 2 == 1 ||
      stringr::str_length(to) %% 2 == 1) {
    stop(
      "The values for 'from' and 'to' must be strings in the format 'YYYY', 'YYYYMM' or 'YYYYMMDD'."
    )
  }

  if (stringr::str_length(from) == 6) {
    from_date <- paste0(from, "01")
  } else if (stringr::str_length(from) == 4) {
    from_date <- paste0(from, "0101")
  } else {
    from_date <- from
  }

  if (stringr::str_length(to) == 8) {
    to_date <- to
  } else {
    if (stringr::str_length(to) == 6) {
      to <- paste0(to, "01")
    } else if (stringr::str_length(to) == 4) {
      to <- paste0(to, "1231")
    }

    to_date <- lubridate::ymd(to) %>%
      lubridate::ceiling_date(unit = "month") - 1

    to_date <- gsub("-", "", as.character(to_date))
  }

  if (as.numeric(to_date) < as.numeric(from_date)) {
    stop("'from' must be earlier than or equal to 'to'")
  }

  # Station validation - check whether either of the selectable stations was chosen, 260 and 275 in case of Veluwe 
  selectable_stations <- c("260", "275", "209", "210", "215", "225", "229", "235", "240", "242", "248", "249", "251", "257", "258", "260", "265", 
                           "267", "269", "270", "273", "275", "277", "278", "279", "280", "283", "285", "286", "290", "308", "310", "311", "312", 
                           "313", "315", "316", "319", "323", "324", "330", "331", "340", "343", "344", "348", "350", "356", "370", "375", "377",
                           "380", "391", "392")
  
    stationID <- as.character(stationID)

  if (!stationID %in% selectable_stations) {
    stop(
      paste0(
        "Invalid stationID. Choose one of: ",
        paste(selectable_stations, collapse = ", ")
      )
    )
  }

#Build URL to retrieve the data from the KNMI website 
baseURL <- "https://www.daggegevens.knmi.nl/klimatologie/daggegevens"
  URL <- paste0(
    baseURL,
    "?start=", from_date,
    "&end=", to_date,
    "&stns=", stationID,
    "&ALL"
  )

  #Download data from KNMI 
  data_spinup <- readr::read_csv(
    URL,
    col_names = FALSE,
    comment = "#",
    show_col_types = FALSE
  ) %>%
    dplyr::as_tibble()

  colnames(data_spinup) <- c(
    "STN", "YYYYMMDD", "FG", "TG", "TN",
    "TX", "RH", "DDVEC", "UG", "UX", "UN"
  )

  # Select only requested variables
  data_spinup <- data_spinup %>%
    tidyr::drop_na(dplyr::all_of(vars)) %>%
    select(dplyr::all_of(c("YYYYMMDD", vars)))

  # Scaling because KNMI works in 0.1
  if ("FG" %in% vars) data_spinup$FG <- data_spinup$FG / 10
  if ("TG" %in% vars) data_spinup$TG <- data_spinup$TG / 10
  if ("TN" %in% vars) data_spinup$TN <- data_spinup$TN / 10
  if ("TX" %in% vars) data_spinup$TX <- data_spinup$TX / 10
  if ("RH" %in% vars) data_spinup$RH <- data_spinup$RH / 10

  return(data_spinup)
}

#Run function for spin up KNMI data 
KNMI_spinup_datafile <- ""
KNMI_spinup_datafile <- get_spinup_data(
  stationID = param_stationID_spinup,
  from      = param_from_date_spinup,
  to        = param_to_date_spinup,
  vars      = param_vars_spinup
)

In [10]:
#KNMI data preprocessor
#Reordering the date from YYYYMMDD to Year, Month, and Day 
Preprocessed_KNMI_daily_datafile <- KNMI_daily_datafile %>%
  mutate(YYYYMMDD = as.character(YYYYMMDD)) %>%
  mutate(
    date  = ymd(YYYYMMDD),
    Year  = year(date),
    Month = month(date),
    Day   = day(date)
  ) %>%
  select(-YYYYMMDD, -date) %>%
#Renaming KNMI variables to match LANDIS-II names   
  rename(any_of(c(
    windSpeed = "FG",
    temp      = "TG",
    mintemp   = "TN",
    maxtemp   = "TX",
    precip    = "RH",
    RH        = "UG",
    maxRH     = "UX",
    minRH     = "UN"
  ))) %>%
 #Reorganize and rename columns 
pivot_longer(
    cols = -c(Year, Month, Day),
    names_to = "Variable",
    values_to = "Value"
  ) %>%
  select(Year, Month, Day, Variable, Value)

## KNMI spinup data ##
#Reordering the date from YYYYMMDD to Year, Month, and Day 
Preprocessed_KNMI_spinup_datafile <- KNMI_spinup_datafile %>%
  mutate(YYYYMMDD = as.character(YYYYMMDD)) %>%
  mutate(
    date  = ymd(YYYYMMDD),
    Year  = year(date),
    Month = month(date),
    Day   = day(date)
  ) %>%
  select(-YYYYMMDD, -date) %>%
#Renaming KNMI variables to match LANDIS-II names   
  rename(any_of(c(
    windSpeed = "FG",
    temp      = "TG",
    mintemp   = "TN",
    maxtemp   = "TX",
    precip    = "RH",
    RH        = "UG",
    maxRH     = "UX",
    minRH     = "UN"
  ))) %>%
 #Reorganize and rename columns 
  pivot_longer(
    cols = -c(Year, Month, Day),
    names_to = "Variable",
    values_to = "Value"
  ) %>%
  select(Year, Month, Day, Variable, Value)

# Save output in CSV files (?)
write_csv(Preprocessed_KNMI_daily_datafile, "KNMI_daily.csv")
write_csv(Preprocessed_KNMI_spinup_datafile, "KNMI_spinup.csv")

In [ ]:
# Species trait data retriever
Species_trait_datafile <- "Species_trait_datafile"

In [ ]:
# Species trait data preprocessor
Preprocessed_species_trait_datafile <- ""
Preprocessed_species_trait_datafile <- Species_trait_datafile

In [ ]:
# Initial community data retriever
Initial_community_datafile <- "Initial community datafile"

In [ ]:
# Initial community data proprocessor
Preprocessed_initial_community_datafile <- ""
Preprocessed_initial_community_datafile <- Initial_community_datafile

In [ ]:
# Scenario and ecoregions builder
print(Preprocessed_KNMI_daily_datafile)
print(Preprocessed_KNMI_spinup_datafile
print(Preprocessed_species_trait_datafile)
print(Preprocessed_initial_community_datafile)
print(Preprocessed_KNMI_spinup_datafile)
Scenario_file <- "scenario file"

In [ ]:
# LANDIS II runner
print(Scenario_file)